<a href="https://colab.research.google.com/github/geocarvalho/uni-proj/blob/master/IN1164/project_1/PSO_and_ABC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implementação: 
- PSO padrão e uma variação.
- ABC (ou Firefly) padrão e uma variação.


Testar em 6 funções de benchmark com variação de atributos de 10, 20 e 50.
Funções:
- [Ackley function](http://benchmarkfcns.xyz/benchmarkfcns/ackleyfcn.html) - [em python](https://www.cs.unm.edu/~neal.holts/dga/benchmarkFunction/ackley.html)
- [Alpine function](http://benchmarkfcns.xyz/benchmarkfcns/alpinen1fcn.html)
- [Schwefel Function](http://benchmarkfcns.xyz/benchmarkfcns/schwefelfcn.html) - [em python](https://www.cs.unm.edu/~neal.holts/dga/benchmarkFunction/schwefel.html)
- [Happy Cat Function](http://benchmarkfcns.xyz/benchmarkfcns/happycatfcn.html) 
- [Brown function](http://benchmarkfcns.xyz/benchmarkfcns/brownfcn.html)
- [Exponential function](http://benchmarkfcns.xyz/benchmarkfcns/exponentialfcn.html)

Para funções olhar o site: http://benchmarkfcns.xyz/

# Funções de fitness


In [2]:
import math
import random
import numpy as np

def simple_func(position):
  """ Simple function that models the problem """
  return position[0]**2 + position[1]**2 + 1

def sphere_func(positions):
  """ Sphere function 
  http://benchmarkfcns.xyz/benchmarkfcns/spherefcn.html"""
  c = np.array(positions)
  x = c**2
  return np.sum(x)

def ackley_func(positions):
  """ Ackley function"""
  c = np.array(positions)
  firstSum = np.sum(c**2.0)
  secondSum = np.sum(np.cos(2.0*math.pi*c))
  n = float(len(positions))
  return -20.0*math.exp(-0.2*math.sqrt(firstSum/n)) - math.exp(secondSum/n) + 20 + math.e

def alpine_func(positions):
  """ Alpine function """
  c = np.array(positions)
  scores = np.sum(abs(c * np.sin(c) + 0.1*c))
  return scores

def schwefel_func(positions):
  """F7 Schwefel's function
  multimodal, asymmetric, separable"""
  c = np.array(positions)
  n = len(c)
  alpha = 418.982887
  fitness = np.sum(c * np.sin(np.sqrt(abs(c))))
  return alpha * n - fitness

def happy_cat_func(positions):
  """ Happy Cat function """
  alpha = 0.5
  c = np.array(positions)
  n = len(c)
  x2 = np.sum(c*c)
  scores = (((x2 - n)**2)**alpha + (0.5*x2 + np.sum(c)))/ (n + 0.5)
  return scores

def brown_func(positions):
  """ Brown function """
  c = np.array(positions)
  n = len(c)
  x = c**2
  scores = 0
  for i in range(n-1):
    scores = scores + x[i]**(x[i+1] + 1) + x[i+1]**(x[i]+1)
  return scores

def exponential_func(positions):
  """ Exponential function """
  c = np.array(positions)
  x2 = c**2
  scores = -np.exp(-0.5 * np.sum(x2))
  return scores


# PSO padrão

* [1995 Particle swarm optimization](https://ieeexplore.ieee.org/abstract/document/488968)
* [1998 A Modified Particle Swarm Optimizer](https://ieeexplore.ieee.org/document/699146)

In [ ]:
def PSO(problem, dimension, var_min, var_max, n_iterations, n_particles, 
        w, c1, c2, show_iter):
  """ PSO algorithm """
  # Initialization
  # np.seterr(over='ignore')
  particle_position_vector = np.random.uniform(var_min,var_max,(
      n_particles, dimension))
  pbest_position = np.copy(particle_position_vector)
  pbest_fitness_value = np.full(shape=n_particles, fill_value=float('inf'))
  gbest_fitness_value = float('inf')
  gbest_position = np.full(shape=dimension, fill_value=0)
  velocity_vector = np.zeros(shape=(n_particles, dimension))
  iteration = 0
  # Start iterations
  while iteration < n_iterations:
    for p in range(n_particles):
      fitness_candidate = problem(particle_position_vector[p])
      
      # Calculate pbest
      if pbest_fitness_value[p] > fitness_candidate:
        pbest_fitness_value[p] = fitness_candidate
        pbest_position[p] = np.copy(particle_position_vector[p])
    
    # Update velocity of each particle
    for p in range(n_particles):
      new_velocity = (w * velocity_vector[p]) + \
      ((c1 * random.random()) * (pbest_position[p] - particle_position_vector[p])) + \
      ((c2 * random.random()) * (gbest_position - particle_position_vector[p]))
      new_position = new_velocity + particle_position_vector[p]
      # Check if the positions is var_min<x<var_max
      for value in new_position:
        index = list(new_position).index(value)
        new_position[index] = np.max([var_min, value])
        new_position[index] = np.min([var_max, new_position[index]])
      particle_position_vector[p] = new_position
    
    # Calculate gbest
    gbest_candidate = np.min(pbest_fitness_value)
    index_gbest = list(pbest_fitness_value).index(gbest_candidate)
    if gbest_fitness_value > gbest_candidate:
      gbest_fitness_value = gbest_candidate
      gbest_position = np.copy(pbest_position[index_gbest])
    if show_iter:
      print(gbest_fitness_value, " :", gbest_position)

    iteration += 1

  return gbest_position, gbest_fitness_value

In [ ]:
# Problem definition
fitness_functions = {"Ackley function": ackley_func, 
                     "Alpine function": alpine_func, 
                     "Schwefel's function": schwefel_func, 
                     "Happy Cat function": happy_cat_func,
                     "Brown function": brown_func,
                     "Exponential function": exponential_func}
dimensions = [10, 20, 50]

for dim in dimensions:
  for name, func in fitness_functions.items():
    kwargs = {"problem": func, "dimension": dim, "var_min": -10, 
              "var_max": 10, "n_iterations": 50, "n_particles": 10,
              "w": 0.8, "c1": 1.5, "c2": 1.5, "show_iter": False}
    gbest_pos, gbest_value = PSO(**kwargs)
    print("dimension: %s,  %s, gbest: %s" % (
        dim, name, gbest_value))

dimension: 10,  Ackley function, gbest: 4.4651535393709025
dimension: 10,  Alpine function, gbest: 0.5057087235238676
dimension: 10,  Schwefel's function, gbest: 4172.8414280022325
dimension: 10,  Happy Cat function, gbest: 0.18565262411178585
dimension: 10,  Brown function, gbest: 0.8822726002624524
dimension: 10,  Exponential function, gbest: -0.749209113469025
dimension: 20,  Ackley function, gbest: 1.3297720103433268
dimension: 20,  Alpine function, gbest: 0.07231594251762728
dimension: 20,  Schwefel's function, gbest: 8362.264817981892
dimension: 20,  Happy Cat function, gbest: 0.3500924647481427
dimension: 20,  Brown function, gbest: 3.445086310107728
dimension: 20,  Exponential function, gbest: -0.019191816755387215
dimension: 50,  Ackley function, gbest: 2.303687092694862
dimension: 50,  Alpine function, gbest: 1.6362777041398016
dimension: 50,  Schwefel's function, gbest: 20891.600416242938
dimension: 50,  Happy Cat function, gbest: 0.6109132173414291
dimension: 50,  Brown fun

# Variação do PSO

* [2011 A novel particle swarm optimization algorithm with adaptive inertia weight](https://www.sciencedirect.com/science/article/abs/pii/S156849461100055X)

## A proposta de calcular o peso inercial

* O modelo de peso inercial proposto é baseado na adaptação do peso de inércia para o problema. Para isso ele determina a porcentagem de sucesso;

* Um alto valor de porcentagem de sucesso indica que as partículas convergiram para um ponto que é longe do ótimo e todo o enxame está se movendo lentamente em direção ao ótimo. Enquanto uma baixa porcentagem indica que as partículas estão ocilando ao redor do ótimo sem evolução.

$P_s(t) = \frac{\sum^n_{i=0}S(i,t)}{n}$

* Onde $n$ é o número de partículas, $P_s \in [0,1]$ é a porcentagem de partículas que tiveram uma melhora no seu *fitness* na última iteração. $\sum^n_{i=1}S(i,t)$ são todas as partículas que tiveram sucesso em minimizar o problema.  

In [21]:
def AIWPSO(problem, dimension, var_min, var_max, n_iterations, n_particles, 
        w, c1, c2, w_max, w_min, show_iter):
  """ PSO algorithm """
  # Initialization
  # np.seterr(over='ignore')
  particle_position_vector = np.random.uniform(var_min,var_max,(
      n_particles, dimension))
  pbest_position = np.copy(particle_position_vector)
  pbest_fitness_value = np.full(shape=n_particles, fill_value=float('inf'))
  gbest_fitness_value = float('inf')
  gbest_position = np.full(shape=dimension, fill_value=0)
  velocity_vector = np.zeros(shape=(n_particles, dimension))
  iteration = 0
  # Start iterations
  while iteration < n_iterations:
    sucess_count = 0
    for p in range(n_particles):
      fitness_candidate = problem(particle_position_vector[p])
      
      # Calculate pbest
      if pbest_fitness_value[p] > fitness_candidate:
        sucess_count += 1
        pbest_fitness_value[p] = fitness_candidate
        pbest_position[p] = np.copy(particle_position_vector[p])
    
    # Update velocity of each particle
    for p in range(n_particles):
      new_velocity = (w * velocity_vector[p]) + \
      ((c1 * random.random()) * (pbest_position[p] - particle_position_vector[p])) + \
      ((c2 * random.random()) * (gbest_position - particle_position_vector[p]))
      new_position = new_velocity + particle_position_vector[p]
      # Check if the positions is var_min<x<var_max
      for value in new_position:
        index = list(new_position).index(value)
        new_position[index] = np.max([var_min, value])
        new_position[index] = np.min([var_max, new_position[index]])
      particle_position_vector[p] = new_position
    
    # Calculate gbest
    gbest_candidate = np.min(pbest_fitness_value)
    index_gbest = list(pbest_fitness_value).index(gbest_candidate)
    if gbest_fitness_value > gbest_candidate:
      gbest_fitness_value = gbest_candidate
      gbest_position = np.copy(pbest_position[index_gbest])
    if show_iter:
      print(gbest_fitness_value, " :", gbest_position)
    # Inertia weight adaptation
    ps = sucess_count / n_particles
    w = (w_max - w_min)*ps + w_min
    iteration += 1

  return gbest_position, gbest_fitness_value

In [22]:
# Problem definition
fitness_functions = {"Ackley function": ackley_func, 
                     "Alpine function": alpine_func, 
                     "Schwefel's function": schwefel_func, 
                     "Happy Cat function": happy_cat_func,
                     "Brown function": brown_func,
                     "Exponential function": exponential_func}
dimensions = [10, 20, 50]

for dim in dimensions:
  for name, func in fitness_functions.items():
    kwargs = {"problem": func, "dimension": dim, "var_min": -10, 
              "var_max": 10, "n_iterations": 50, "n_particles": 10,
              "w": 0.8, "c1": 1.5, "c2": 1.5, "w_max": 1, "w_min":0,
              "show_iter": False}
    gbest_pos, gbest_value = AIWPSO(**kwargs)
    print("dimension: %s,  %s, gbest: %s" % (
        dim, name, gbest_value))

dimension: 10,  Ackley function, gbest: 0.09884467708461431
dimension: 10,  Alpine function, gbest: 0.031257931387239826
dimension: 10,  Schwefel's function, gbest: 4170.466759211715
dimension: 10,  Happy Cat function, gbest: 0.12563134978262575
dimension: 10,  Brown function, gbest: 0.001460073635590665
dimension: 10,  Exponential function, gbest: -0.42022199240026076
dimension: 20,  Ackley function, gbest: 3.199152109167564
dimension: 20,  Alpine function, gbest: 1.1124322954211165
dimension: 20,  Schwefel's function, gbest: 8353.834850899804
dimension: 20,  Happy Cat function, gbest: 0.04400901747846818
dimension: 20,  Brown function, gbest: 31.131535962926925
dimension: 20,  Exponential function, gbest: -0.0005908147897128343
dimension: 50,  Ackley function, gbest: 3.8544211521038716
dimension: 50,  Alpine function, gbest: 19.272072935373703
dimension: 50,  Schwefel's function, gbest: 20909.554973409333
dimension: 50,  Happy Cat function, gbest: 0.19924454270898026
dimension: 50,  

# ABC padrão

* [2005 An idea based on honey bee swarm for numerical optimization](http://citeseerx.ist.psu.edu/viewdoc/summary?doi=10.1.1.714.4934)
* [2015 PS–ABC: A hybrid algorithm based on particle swarm and artificial bee colony for high-dimensional optimization problems](https://www.sciencedirect.com/science/article/abs/pii/S0957417415005035)

In [17]:
def fitness_calc(value):
  """ Calculate the fitness based on the object function result """
  if value >= 0:
    return 1/(1+value)
  else:
    return 1 + abs(value)

def generate_new_solution(dimension, food_source, num, food, fit, 
                          var_min, var_max, trial, fitness_func,
                          population, obj):
  """ Generates a new solution """
  # Phi is a random number between -1 and 1
  phi = random.uniform(-1,1)
  ## Select a random variable to change
  var_index_lst = list(range(dimension))
  var_index = random.choice(var_index_lst)
  food_var = food[var_index]
  ## Select a random partner
  partner_index_lst = list(range(food_source))
  partner_index_lst.remove(num)
  partner_index = random.choice(partner_index_lst)
  partner = population[partner_index]
  partner_var = partner[var_index]
  ## Create a new food location
  x_new = food_var + (phi*(food_var - partner_var))
  ## Check if the new location violates the lower bound (0<=x<=10)
  x_new = np.max([var_min, x_new])
  x_new = np.min([var_max, x_new])
  copy_food = np.copy(food)
  copy_food[var_index] = x_new
  ## Evaluate the fitness
  new_obj = fitness_func(copy_food)
  new_fit = fitness_calc(new_obj)
  ## Perform greedy selection
  if new_fit > fit[num]:
    food[var_index] = x_new
    fit[num] = new_fit
    obj[num] = new_obj
    trial[num] = 0
  else:
    trial[num] += 1
  return food, obj, fit, trial

def ABC(dimension, fitness_func, var_min, var_max, pop_size, limit,
        n_cycles, show_iter):
  # Parameters
  employed_bees = int(pop_size/2)
  onlooker_bees = int(pop_size/2)
  food_source = int(pop_size/2)

  # Initialization
  ## The food source population with 4 variables, create random value 0<x<10
  population = np.random.uniform(var_min,var_max,(food_source, dimension))
  ## Calculate the object function for each food source
  obj = np.apply_along_axis(fitness_func, 1, population)
  ## Calculate randintfitness of the population
  fit = np.array([fitness_calc(value) for value in obj])
  ## Generate initial trial vector
  trial = np.zeros(food_source)
  ## Start the best solution variable
  b_fit_solution = np.max(fit)
  b_index = list(fit).index(b_fit_solution)
  b_food = population[b_index]
  b_obj = obj[b_index]
  b_cycle = 0
  for cycle in range(n_cycles):
    # Employed bee phase
    for num1, food in enumerate(population, start=0):
      food, obj, fit, trial = generate_new_solution(
          dimension, food_source, num1, food, fit, var_min, var_max,
          trial, fitness_func, population, obj)
    # Calculate the probability values
    max = np.max(fit)
    prob = fit/np.sum(fit) #((fit/max)*0.9)+0.1
    # Onlooker bee phase
    num2 = 0
    for obee in range(onlooker_bees):
      search_for_food = True
      while search_for_food:
        # Control the flow to search for food using num and food
        if num2 == len(population-1):
          num2 = 0
        food = population[num2]
        ## Select a random number
        r = random.uniform(0,1)
        if r <= prob[num2]:
          food, obj, fit, trial = generate_new_solution(
              dimension, food_source, num2, food, fit, var_min, var_max,
              trial, fitness_func, population, obj)
          num2 += 1
          search_for_food = False
        else:
          num2 += 1
    # Memorize the best solution
    candidate = np.max(fit)
    if candidate > b_fit_solution:
      b_index = list(fit).index(candidate)
      b_food = np.copy(population[b_index])
      b_fit_solution = fit[b_index]
      b_obj = obj[b_index]
      b_cycle = cycle
      if show_iter:
        print("____Inter values")
        print(population)
        print(obj)
        print(fit)
        print(trial)
    # Scout bee phase
    ## Select one solution for which the trial is greater than limit
    trial_val = np.max(trial)
    num3 = list(trial).index(trial_val)
    if trial_val > limit:
      # Generate a new random solution
      # same as = var_min + (var_max - var_min)*random.uniform(0,1) for each index
      population[num3] = np.random.uniform(var_min,var_max,(dimension))
      obj[num3] = fitness_func(population[num3])
      fit[num3] = fitness_calc(obj[num3])
      trial[num3] = 0
  if show_iter:
    print("\n____Final values")
    print(population)
    print(obj)
    print(fit)
    print(trial)
  return b_food, b_obj, b_fit_solution, b_cycle


In [18]:
# ABC Problem

fitness_functions = {"Ackley function": ackley_func, 
                     "Alpine function": alpine_func, 
                     "Schwefel's function": schwefel_func, 
                     "Happy Cat function": happy_cat_func,
                     "Brown function": brown_func,
                     "Exponential function": exponential_func}
dimensions = [10, 20, 50]

for dim in dimensions:
  for name, func in fitness_functions.items():
    kwargs = {"dimension": dim, "fitness_func": func, "var_min": 0.0, 
              "var_max": 10.0, "pop_size": 10, "limit": 5,
              "n_cycles": 50, "show_iter": False}
    b_food, b_obj, b_fit_solution, b_cycle = ABC(**kwargs)
    print("dimension: %s, %s, object: %s, fitness: %s, cycle: %s" % (
            dim, name, b_obj, b_fit_solution, b_cycle))

# print("\n____Best solution:")
# print("best positions: ", b_food)
# print("best object: ", b_obj)
# print("best fitness: ",b_fit_solution)
# print("best cycle: ", b_cycle)

dimension: 10, Ackley function, object: 5.410246794075839, fitness: 0.15600023401972143, cycle: 22
dimension: 10, Alpine function, object: 5.740339371268815, fitness: 0.14836048230191085, cycle: 30
dimension: 10, Schwefel's function, object: 4151.253320831388, fitness: 0.00024083309054944034, cycle: 48
dimension: 10, Happy Cat function, object: 2.0063681867332908, fitness: 0.3326272558407414, cycle: 38
dimension: 10, Brown function, object: 96.75365431037886, fitness: 0.010229796594865777, cycle: 26
dimension: 10, Exponential function, object: -2.131372676862392e-100, fitness: 1.0, cycle: 0
dimension: 20, Ackley function, object: 11.064184871473664, fitness: 0.08288997645954078, cycle: 49
dimension: 20, Alpine function, object: 14.94750190689597, fitness: 0.06270574575492498, cycle: 21
dimension: 20, Schwefel's function, object: 8306.943886988664, fitness: 0.00012036672534176982, cycle: 40
dimension: 20, Happy Cat function, object: 10.756932093699053, fitness: 0.08505620276023663, cycl

# Variação do ABC

* [2010 Gbest-guided artificial bee colony algorithm for numerical function optimization](https://www.sciencedirect.com/science/article/abs/pii/S0096300310009136)

## Substituição da equação de atualização de posição

* No algortimo padrão para atualizar a posição da comida pelas abelhas se usa a fórmula:

$v_{ij} = x_{ij} + \phi_{ij}(x_{ij} - x_{kj})$

* No artigo eles adicionam um termo *gbest* no final:

$v_{ij} = x_{ij} + \phi_{ij}(x_{ij} - x_{kj}) + \psi(y_j - x_{kj})$

* Onde $y_j$ é o *j* elemento do melhor solução global e $\psi_{ij}$ é um número aleatório entre $[0,C]$, sendo $C$ uma constante não negativa.
> O melhor valor de $C$ de acordo com os experimentos dos autores foi de $C=1.5$

In [19]:
def fitness_calc(value):
  """ Calculate the fitness based on the object function result """
  if value >= 0:
    return 1/(1+value)
  else:
    return 1 + abs(value)

def gabc_generate_new_solution(dimension, food_source, num, food, fit, 
                          var_min, var_max, trial, fitness_func,
                          population, obj, b_food):
  """ Generates a new solution """
  # Phi is a random number between -1 and 1
  phi = random.uniform(-1,1)
  ## Select a random variable to change
  var_index_lst = list(range(dimension))
  var_index = random.choice(var_index_lst)
  food_var = food[var_index]
  ## Select a random partner
  partner_index_lst = list(range(food_source))
  partner_index_lst.remove(num)
  partner_index = random.choice(partner_index_lst)
  partner = population[partner_index]
  partner_var = partner[var_index]
  ## Create a new food location
  ### For the new equation C=1.5
  c = 1.5
  psi = random.uniform(0,c)
  gbest = b_food[var_index]
  x_new = food_var + (phi*(food_var - partner_var)) + (psi*(gbest - food_var)) 
  ## Check if the new location violates the lower bound (0<=x<=10)
  x_new = np.max([var_min, x_new])
  x_new = np.min([var_max, x_new])
  copy_food = np.copy(food)
  copy_food[var_index] = x_new
  ## Evaluate the fitness
  new_obj = fitness_func(copy_food)
  new_fit = fitness_calc(new_obj)
  ## Perform greedy selection
  if new_fit > fit[num]:
    food[var_index] = x_new
    fit[num] = new_fit
    obj[num] = new_obj
    trial[num] = 0
  else:
    trial[num] += 1
  return food, obj, fit, trial

def GABC(dimension, fitness_func, var_min, var_max, pop_size, limit,
        n_cycles, show_iter):
  # Parameters
  employed_bees = int(pop_size/2)
  onlooker_bees = int(pop_size/2)
  food_source = int(pop_size/2)

  # Initialization
  ## The food source population with 4 variables, create random value 0<x<10
  population = np.random.uniform(var_min,var_max,(food_source, dimension))
  ## Calculate the object function for each food source
  obj = np.apply_along_axis(fitness_func, 1, population)
  ## Calculate randintfitness of the population
  fit = np.array([fitness_calc(value) for value in obj])
  ## Generate initial trial vector
  trial = np.zeros(food_source)
  ## Start the best solution variable
  b_fit_solution = np.max(fit)
  b_index = list(fit).index(b_fit_solution)
  b_food = population[b_index]
  b_obj = obj[b_index]
  b_cycle = 0
  for cycle in range(n_cycles):
    # Employed bee phase
    for num1, food in enumerate(population, start=0):
      food, obj, fit, trial = gabc_generate_new_solution(
          dimension, food_source, num1, food, fit, var_min, var_max,
          trial, fitness_func, population, obj, b_food)
    # Calculate the probability values
    max = np.max(fit)
    prob = fit/np.sum(fit) #((fit/max)*0.9)+0.1
    # Onlooker bee phase
    num2 = 0
    for obee in range(onlooker_bees):
      search_for_food = True
      while search_for_food:
        # Control the flow to search for food using num and food
        if num2 == len(population-1):
          num2 = 0
        food = population[num2]
        ## Select a random number
        r = random.uniform(0,1)
        if r <= prob[num2]:
          food, obj, fit, trial = gabc_generate_new_solution(
              dimension, food_source, num2, food, fit, var_min, var_max,
              trial, fitness_func, population, obj, b_food)
          num2 += 1
          search_for_food = False
        else:
          num2 += 1
    # Memorize the best solution
    candidate = np.max(fit)
    if candidate > b_fit_solution:
      b_index = list(fit).index(candidate)
      b_food = np.copy(population[b_index])
      b_fit_solution = fit[b_index]
      b_obj = obj[b_index]
      b_cycle = cycle
      if show_iter:
        print("____Inter values")
        print(population)
        print(obj)
        print(fit)
        print(trial)
    # Scout bee phase
    ## Select one solution for which the trial is greater than limit
    trial_val = np.max(trial)
    num3 = list(trial).index(trial_val)
    if trial_val > limit:
      # Generate a new random solution
      # same as = var_min + (var_max - var_min)*random.uniform(0,1) for each index
      population[num3] = np.random.uniform(var_min,var_max,(dimension))
      obj[num3] = fitness_func(population[num3])
      fit[num3] = fitness_calc(obj[num3])
      trial[num3] = 0
  if show_iter:
    print("\n____Final values")
    print(population)
    print(obj)
    print(fit)
    print(trial)
  return b_food, b_obj, b_fit_solution, b_cycle


In [20]:
# GABC Problem

fitness_functions = {"Ackley function": ackley_func, 
                     "Alpine function": alpine_func, 
                     "Schwefel's function": schwefel_func, 
                     "Happy Cat function": happy_cat_func,
                     "Brown function": brown_func,
                     "Exponential function": exponential_func}
dimensions = [10, 20, 50]

for dim in dimensions:
  for name, func in fitness_functions.items():
    kwargs = {"dimension": dim, "fitness_func": func, "var_min": 0.0, 
              "var_max": 10.0, "pop_size": 10, "limit": 5,
              "n_cycles": 50, "show_iter": False}
    b_food, b_obj, b_fit_solution, b_cycle = ABC(**kwargs)
    print("dimension: %s, %s, object: %s, fitness: %s, cycle: %s" % (
            dim, name, b_obj, b_fit_solution, b_cycle))

# print("\n____Best solution:")
# print("best positions: ", b_food)
# print("best object: ", b_obj)
# print("best fitness: ",b_fit_solution)
# print("best cycle: ", b_cycle)

dimension: 10, Ackley function, object: 7.855955760465335, fitness: 0.11291835992047178, cycle: 28
dimension: 10, Alpine function, object: 4.323095993434921, fitness: 0.18786059865035679, cycle: 35
dimension: 10, Schwefel's function, object: 4150.784821295955, fitness: 0.00024086026685936384, cycle: 34
dimension: 10, Happy Cat function, object: 0.927522463328447, fitness: 0.5188006983188146, cycle: 43
dimension: 10, Brown function, object: 81.84939443861681, fitness: 0.012070094256885617, cycle: 49
dimension: 10, Exponential function, object: -4.180568005908497e-24, fitness: 1.0, cycle: 0
dimension: 20, Ackley function, object: 11.558561793612908, fitness: 0.07962695222860508, cycle: 48
dimension: 20, Alpine function, object: 9.907823489363272, fitness: 0.0916773177504336, cycle: 49
dimension: 20, Schwefel's function, object: 8311.125258699465, fitness: 0.00012030617548182406, cycle: 43
dimension: 20, Happy Cat function, object: 12.682342759943237, fitness: 0.0730868987530136, cycle: 4

# Outras referências
* [Lec 10 : Particle Swarm Optimization](https://www.youtube.com/watch?v=UKx76ThMXDM)
* [Lec 11 : Implementation of Particle Swarm Optimization using MATLAB](https://www.youtube.com/watch?v=2VJl37RWvkw)
* [Lec 18 : Working of Artificial Bee Colony Algorithm](https://youtu.be/MAhlPwK4_fI)
* [Lec 19 : Implementation of Artificial Bee Colony using MATLAB](https://www.youtube.com/watch?v=2Huy72h7Y20)
* [2003 The particle swarm optimization algorithm: convergence analysis and parameter selection](https://www.sciencedirect.com/science/article/abs/pii/S0020019002004477)
* [2011 A novel particle swarm optimization algorithm with adaptive inertia weight](https://www.sciencedirect.com/science/article/abs/pii/S156849461100055X)
* [2012 A modified Artificial Bee Colony algorithm for real-parameter optimization](https://www.sciencedirect.com/science/article/pii/S0020025510003336)
* [2013 An Overview of Particle Swarm Optimization Variants](https://www.sciencedirect.com/science/article/pii/S1877705813001823)
* [2014 Hierarchical Artificial Bee Colony Algorithm for RFID Network Planning Optimization](https://www.hindawi.com/journals/tswj/2014/941532/)
* [2014 Hybrid Artificial Bee Colony Algorithm and Particle Swarm Search for Global Optimization](https://www.hindawi.com/journals/mpe/2014/832949/)
* [2015 PS–ABC: A hybrid algorithm based on particle swarm and artificial bee colony for high-dimensional optimization problems](https://www.sciencedirect.com/science/article/abs/pii/S0957417415005035)
* [2019 An adaptive encoding learning for artificial bee colony algorithms](https://www.sciencedirect.com/science/article/abs/pii/S1877750318310123#:~:text=An%20adaptive%20encoding%20learning%20for%20artificial%20bee%20colony%20algorithms%20is,solve%20global%20numerical%20optimization%20problems.&text=An%20adaptive%20selection%20mechanism%20based,the%20exploration%20and%20exploitation%20abilities.)
* [2020 Population size in Particle Swarm Optimization](https://www.sciencedirect.com/science/article/pii/S2210650220303710)